In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [20]:
class Camera:
    """
    Camera class for simulating each camera in a scanning setup.
    """
    def __init__(self, width, height, f, pixel_size):
        self.width = width              # width of the camera sensor in mm
        self.height = height            # height of the camera sensor in mm
        self.aspect = width/height
        self.pixel_size = pixel_size
        self.f = f                      # focal length of the camera lens
        self.K = np.array([[f, 0, width/2],
                           [0, f, height/2],
                           [0, 0, 1]])
        self.matrix = np.array([0,0,0])

    def capture(self, pose: list):
        """
        pose: [x_pos, y_pos, z_pos, roll_deg, pitch_deg, yaw_deg] 
        """
        # Extrinsic parameters (rotation and translation)     
        R = rotation_matrix_from_euler(pose[3], pose[4], pose[5])   # Identity rotation (camera aligned with world axes)
        t = np.array([pose[0], pose[1], pose[2]])  # Camera positioned 5 units away along the Z-axis
        
        # Combine extrinsics into 3x4 matrix
        Rt = np.hstack((R, t))
               
        # Projection matrix: P = K [R | t]
        P = self.K @ Rt
        
        # Project the point
        X_image = P @ X_world
        
        # Perspective division to get pixel coordinates
        x_pixel = X_image[0, 0] / X_image[2, 0]
        y_pixel = X_image[1, 0] / X_image[2, 0]
        
        print(f"Projected pixel coordinates: ({x_pixel:.2f}, {y_pixel:.2f})")
        return img

    def camera_pyramid(self):
        # this is the pyramid shape that is often used to represent a camera and its pose
        # recommend changing the z-distance to be focal lenght and the x,y to be sensor height and width
        pyramid = np.array([[self.coordinate[0], self.coordinate[1], self.coordinate[2]],
                            [self.coordinate[0]-4,self.coordinate[1]+3,self.coordinate[2] + 10],
                            [self.coordinate[0]+4,self.coordinate[1]+3,self.coordinate[2] + 10],
                            [self.coordinate[0]+4,self.coordinate[1]-3,self.coordinate[2] + 10],
                            [self.coordinate[0]-4,self.coordinate[1]-3,self.coordinate[2] + 10]])
        return pyramid
   
    def rotation_matrix_from_euler(roll_deg, pitch_deg, yaw_deg):
        # Convert degrees to radians
        roll = np.radians(roll_deg)
        pitch = np.radians(pitch_deg)
        yaw = np.radians(yaw_deg)
    
        # Rotation around X-axis (Roll)
        R_x = np.array([[1, 0, 0],
                        [0, np.cos(roll), -np.sin(roll)],
                        [0, np.sin(roll),  np.cos(roll)]])
    
        # Rotation around Y-axis (Pitch)
        R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                        [0, 1, 0],
                        [-np.sin(pitch), 0, np.cos(pitch)]])
    
        # Rotation around Z-axis (Yaw)
        R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                        [np.sin(yaw),  np.cos(yaw), 0],
                        [0, 0, 1]])
    
        # Combined rotation: R = Rz * Ry * Rx
        R = R_z @ R_y @ R_x
        return R


In [21]:
class Object:
    """
    Object class to define the object to be scanned. The object is positioned at the origin of the world coordinates
    thus the camera moves about the object. 
    """
    def __init__(self,filename: str):
        if not isinstance(filename, str):
            raise TypeError("The filename must be a string.")

        self.mesh = o3d.io.read_triangle_mesh(filename)

        # Check if the mesh is loaded correctly
        if not self.mesh.is_empty():
            print("STL file loaded successfully!")
        else:
            print("Failed to load STL file.")

        self.pcd = self.mesh.sample_points_uniformly(number_of_points=10000)  # Adjust the number of points as needed
        self.pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))


    def visualize(self):
        # Compute vertex normals for better visualization
        self.mesh.compute_vertex_normals()

        # Visualize the 3D mesh
        o3d.visualization.draw_geometries([self.mesh])

In [23]:
class Laser:
    """
    A laser illumination source that projects a grid of dots on the object. 

    pose: [x_pos, y_pos, z_pos, roll_deg, pitch_deg, yaw_deg] 
    """
    def __init__(self, pose: list, line: bool):
        self.pos = np.array(pose)

        if line:
            self.type = "grid"
        else: 
            self.type = "line"

    def setup_grid(self, grid_size: int, dot_diameter: float, spacing: float):
        if not self.type == "grid":
            raise PermissionError("This method is only available for grid type lasers.")
        self.grid_size = no.array([size,size])
        self.dot_diameter = dot_diameter #mm
        """Creates a uniform grid of laser points."""
        x = np.linspace(-grid_size, grid_size, spacing)
        y = np.linspace(-grid_size, grid_size, spacing)
        xv, yv = np.meshgrid(x, y)
        laser_points = np.column_stack([xv.ravel(), yv.ravel(), np.zeros_like(xv.ravel())])
        return laser_points
    
    def setup_line(self, width: float):
        if not self.type == "line":
            raise PermissionError("This method is only available for line type lasers.")
        self.width = width

    def cast_rays(self):
        pass
        

In [17]:
""" 
Setup code block
"""

# import object
duck_file = os.path.abspath("../../docs/Rubber_Duck.stl")
duck = Object(duck_file)

# create cameras
Lcam = Camera(2560,960, 2.43)
Rcam = Camera(2560,960, 2.43)



STL file loaded successfully!


In [9]:
"""
Scanning code block
"""


'\nScanning code block\n'

In [10]:
"""
Image Processing code block
"""

'\nImage Processing code block\n'

In [11]:
"""
Model Creation code block
"""

'\nModel Creation code block\n'

In [19]:
duck.visualize()
